In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd
import re
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
browser.get('https://www.pisos.com/alquiler_viviendas/#provincias')

# Handle cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#didomi-notice-disagree-button"))
    ).click()
    time.sleep(2)  # Increase sleep time to ensure the click is registered
except Exception as e:
    print(f"Error handling cookies: {e}")

html = browser.page_source
soup = BeautifulSoup(html, 'html.parser')
links = soup.find_all('a', class_='seo-box__location-link seo-box__location-link--level2')

resultados = []
for link in links:
    nombre = link.get_text(strip=True)
    href = link.get('href')
    
    #verificar que el texto tenga un nomero
    if any(char.isdigit() for char in nombre):
        resultados.append({ #cambiar al formato
            'provincia': nombre,
            'enlace': f'https://www.pisos.com{href}/fecharecientedesde-desc/'  
        })

browser.quit()

urls_actualizacion = []

for resultado in resultados:
    provincia = resultado['provincia']
    base_url = resultado['enlace']
    
    browser = webdriver.Chrome()

    no_results_counter = 0  
    enlaces_extraidos = []  

    for page in range(1, 101):  # 100 paginas para ponernos en lo peor
        url = f"{base_url}{page}/"
        print(f"Accediendo a la página {page} de {provincia}: {url}")
        browser.get(url)

        # cookies
        try:
            browser.find_element(by=By.CSS_SELECTOR, value="#didomi-notice-disagree-button").click()
            time.sleep(1)  
        except:
            pass  

        html = browser.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # Verifica si la página tiene resultados
        no_results = soup.find('div', class_='no-results')
        if no_results:
            print(f"No hay resultados en la página {page} de {provincia}.")
            no_results_counter += 1
            if no_results_counter >= 3:
                print(f"Han pasado 3 páginas sin resultados en {provincia}, pasando a la siguiente provincia.")
                break  
            continue  

        no_results_counter = 0

        inmuebles = soup.find_all('div', class_='ad-preview')

        enlaces_filtrados = []

        for inmueble in inmuebles:
            fecha_span = inmueble.find('span', class_='ad-preview__product-tag--date')
            
            if fecha_span:
                fecha_texto = fecha_span.get_text(strip=True).lower()
                if any(palabra in fecha_texto for palabra in ["horas", "minutos", "ayer"]):
                    enlace = inmueble.find('a', class_='ad-preview__title')
                    if enlace and enlace.get('href'):
                        enlaces_filtrados.append(f'https://www.pisos.com{enlace.get("href")}')

        if enlaces_filtrados:
            enlaces_extraidos.extend(enlaces_filtrados)
        else:
            print(f"No se han extraído enlaces en la página {page} de {provincia}. Pasando a la siguiente provincia.")
            break  

    if enlaces_extraidos:
        urls_actualizacion.extend(enlaces_extraidos)

    browser.quit()


print("Enlaces extraídos:")
for url in urls_actualizacion:
    print(url)


KeyboardInterrupt: 

In [15]:
def obtener_detalles_inmueble(session, url_inmueble):
    try:
        response = session.get(url_inmueble)
        response.raise_for_status()  # Verifica si la solicitud fue exitosa
        soup = BeautifulSoup(response.text, "html.parser")

        def get_text(selector, default="Error al acceder al dato"):
            try:
                element = soup.select_one(selector)
                return element.text.strip() if element else default
            except Exception as e:
                print(f"Error al extraer {selector}: {e}")
                return default

        try:
            nombre = get_text("h1.ad-title")
        except Exception as e:
            nombre = "Error al acceder al nombre"
            print(f"Error al extraer nombre: {e}")

        try:
            ubicacion = get_text("div.details__block > p", "Error al acceder a la ubicación")
        except Exception as e:
            ubicacion = "Error al acceder a la ubicación"
            print(f"Error al extraer ubicación: {e}")

        try:
            agencia = get_text("p.owner-info__name a", "Agencia no disponible")
        except Exception as e:
            agencia = "Agencia no disponible"
            print(f"Error al extraer agencia: {e}")

        try:
            precio_m2 = next((item.text.strip() for item in soup.select("ul.features-summary li.features-summary__item") if "€/m²" in item.text), "Error al procesar precio m²")
        except Exception as e:
            precio_m2 = "Error al procesar precio m²"
            print(f"Error al extraer precio m²: {e}")

        try:
            precio_element = soup.select_one("div.price__value.jsPriceValue")
            if precio_element:
                precio = precio_element.text.strip()
                unidad_tiempo_element = soup.select_one("select.price__selector.jsPriceSelector option[selected]")
                if unidad_tiempo_element:
                    unidad_tiempo = unidad_tiempo_element.text.strip()
                    precio_completo = f"{precio} / {unidad_tiempo}"
                else:
                    precio_completo = precio
            else:
                precio_completo = "Error al procesar precio"
        except Exception as e:
            precio_completo = "Error al procesar precio completo"
            print(f"Error al extraer precio: {e}")

        try:
            superficie = get_text("span.features__value", "Error al procesar superficie")
        except Exception as e:
            superficie = "Error al procesar superficie"
            print(f"Error al extraer superficie: {e}")

        try:
            actualizacion = get_text("div.details__block.last-update", "Error al procesar actualización").replace("Última actualización\n", "").strip()
        except Exception as e:
            actualizacion = "Error al procesar actualización"
            print(f"Error al extraer actualización: {e}")

        try:
            certificado_energetico_etiquetas = [etiqueta['class'][1].split('--')[1].strip().upper() for etiqueta in soup.select("span.energy-certificate__tag") if 'energy-certificate__tag--' in etiqueta['class'][1]]
            certificado_energetico = ', '.join(certificado_energetico_etiquetas) if certificado_energetico_etiquetas else "En trámite"
        except Exception as e:
            certificado_energetico = "Error al procesar certificado energético"
            print(f"Error al extraer certificado energético: {e}")

        try:
            detalles = {feature.select_one("span.features__label").text.strip().replace(":", ""): feature.select_one("span.features__value").text.strip() if feature.select_one("span.features__value") else "Si" for feature in soup.select("div.features__feature")}
        except Exception as e:
            detalles = {}
            print(f"Error al extraer detalles: {e}")

        try:
            identificador = re.search(r'-(\d+_\d+)', url_inmueble).group(1) if re.search(r'-(\d+_\d+)', url_inmueble) else 'NaN'
        except Exception as e:
            identificador = 'NaN'
            print(f"Error al extraer identificador: {e}")

        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

        try:
            location_div = soup.select_one("div.location")
            data_params = location_div["data-params"] if location_div else ""
            latitud = re.search(r'latitude=([-0-9.]+)', data_params).group(1) if re.search(r'latitude=([-0-9.]+)', data_params) else 'NaN'
            longitud = re.search(r'longitude=([-0-9.]+)', data_params).group(1) if re.search(r'longitude=([-0-9.]+)', data_params) else 'NaN'
            coordenadas = f"{latitud}, {longitud}"
        except Exception as e:
            coordenadas = "NaN, NaN"
            print(f"Error al extraer coordenadas: {e}")

        return nombre, ubicacion, agencia, precio_m2, precio_completo, superficie, actualizacion, certificado_energetico, detalles, identificador, timestamp, coordenadas
    except Exception as e:
        print(f"Error al procesar el inmueble {url_inmueble}: {e}")
        return ("Error al acceder al inmueble",) * 8 + ({}, "", "", datetime.now().strftime('%Y-%m-%d %H:%M:%S'), "")

In [16]:
def obtener_inmuebles_de_urls(urls_a_scrapear):
    todos_los_inmuebles = []
    columnas = set()
    identificadores_unicos = set()
    session = requests.Session()  # Use a session to maintain persistent connections

    def procesar_url(url_pagina):
        try:
            print(f"Processing URL: {url_pagina}")
            response = session.get(url_pagina)
            if response.status_code == 404:
                print(f"Page not found: {url_pagina}")
                return
            
            soup = BeautifulSoup(response.text, "lxml")
            titulos_soup = soup.find_all("a", class_="ad-preview__title")
            
            for titulo_soup in titulos_soup:
                nombre = titulo_soup.text.strip()
                href = "https://www.pisos.com" + titulo_soup['href']
                print(f"Processing property: {href}")
                nombre_detalle, ubicacion, agencia, precio_m2, precio, superficie, actualizacion, certificado_energetico, detalles, identificador, timestamp, coordenadas = obtener_detalles_inmueble(session, href)
                
                if identificador in identificadores_unicos:
                    print(f"Duplicate property: {identificador}")
                    continue  # Skip if the identifier already exists
                
                identificadores_unicos.add(identificador)
                
                inmueble = {
                    "nombre": nombre,
                    "ubicacion": ubicacion,
                    "agencia": agencia,
                    "precio_m2": precio_m2,
                    "precio": precio,
                    "superficie": superficie,
                    "href": href,
                    "actualizacion": actualizacion,
                    "certificado_energetico": certificado_energetico,
                    "identificador": identificador,
                    "timestamp": timestamp,
                    "coordenadas": coordenadas
                }
                
                inmueble.update(detalles)
                todos_los_inmuebles.append(inmueble)
                columnas.update(inmueble.keys())
        
        except Exception as e:
            print(f"Error processing URL {url_pagina}: {e}")

    with ThreadPoolExecutor(max_workers=10) as executor:
        executor.map(procesar_url, urls_a_scrapear)
    
    print(f"Total properties processed: {len(todos_los_inmuebles)}")
    df_inmuebles = pd.DataFrame(todos_los_inmuebles, columns=list(columnas))
    return df_inmuebles

In [17]:
urls_a_scrapear = extract_property_urls()
df_inmuebles = obtener_inmuebles_de_urls(urls_a_scrapear)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=133.0.6943.142)
Stacktrace:
0   chromedriver                        0x0000000100ae42d4 cxxbridge1$str$ptr + 2739836
1   chromedriver                        0x0000000100adc934 cxxbridge1$str$ptr + 2708700
2   chromedriver                        0x000000010063df90 cxxbridge1$string$len + 93360
3   chromedriver                        0x0000000100617964 chromedriver + 129380
4   chromedriver                        0x00000001006acedc cxxbridge1$string$len + 547836
5   chromedriver                        0x00000001006c5994 cxxbridge1$string$len + 648884
6   chromedriver                        0x0000000100679060 cxxbridge1$string$len + 335232
7   chromedriver                        0x0000000100aacc38 cxxbridge1$str$ptr + 2512864
8   chromedriver                        0x0000000100aaff58 cxxbridge1$str$ptr + 2525952
9   chromedriver                        

In [ ]:
with open('enlaces_actualizacion.txt', 'w') as f:
    for url in urls_actualizacion:
        f.write(url + '\n')

print(f"Hay {len(urls_actualizacion)} enlaces de alquiler, hay que actualizarlos cada dia")